# Homework 2
***
* Assigned: 02/07
* Due: **02/20 at 11:59pm** electronically
* This assignment is worth 100 points.

**Please do read the instructions for each response carefully, since a whole lot of different variety of reponses are involved in the assignment (including pasting code snippets, writing executable code, small writeups). You don't wanna be losing points for silly errors.**

### Jupyter Notes:

* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!
  * you can press shift+enter to execute the code in the cell that your cursor is in.
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_. Please wait for the execution to complete
    * **If the cell is hanging- i.e. running for too long: you can restart the kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute cells from the top
* _Have fun!_


### Setup Your Credentials

Update the following variables

In [ ]:
# Your columbia uni that is used in SSOL
#
# IMPORTANT:  make sure this is consistent with the uni/alias used as your @columbia.edu email in SSOL
#
UNI = "phc2121"

# your instabase username (if you go to the instabase homepage, your username should be in the URL)
USER = "phc2121"

# your repository name containing 
REPO = "CSDS"

# Data Processing

In this lab, you will use various types of tools -- from high-level tools like Data Wrangler to command line tools like `sed` and `awk` -- to perform data parsing and extraction from data encoded into a text file.  The goal of this lab is simply to gain experience with these tools and compare and contrast their usage.

The `lab` directory contains two datasets that you will work with:

1. A dataset of all the movies in 2013 from January to March (`2013films.txt`). It contains Movie name, Production house, Genre, Publisher and some other details.

2. The second dataset (`worldcup.txt`) is a snippet of the following Wikipedia webpage on [FIFA (Soccer) World Cup](http://en.wikipedia.org/wiki/FIFA_World_Cup).
Specifically it is a partially cleaned-up wiki source for the table toward the end of the page that lists teams finishing in the top 4. 

***
# 1. Get Wrangling !!
***
Download the [Trifacta Wrangler](https://www.trifacta.com/products/wrangler/) tool. Load both the datasets into wrangler and try playing around with the tool.

Some tips using Wrangler:

* Check out the introduction [video](https://vimeo.com/19185801) to get a feel of how wrangler works.
* You may wanna start off by loading ~50 lines of data instead of the entire files and play around with the tool.
* Wrangler responds to mouse highlights and clicks on the displayed table cells by suggesting operations on the left sidebar.  
* Hovering over each element shows the result in the table view.  
* Clicking adds the operation.  
* Clear the sidebar by clicking the colored row above the schema row.

## Tasks:

Use Data Wrangler for the following two datasets.  


### 2013films.txt

Use the tool to generate output as follows, i.e., Movie name, Production/Distribtuion house, Director, Genre and publisher. 

        'A dark truth, Magnolia Pictures, Damian Lee, Action,ComingSoon.net
         Table No. 21, Eros International, Aditya Datt, Thriller, BoxOfficeMojo.com
         ...
        
For the purpose of explanation columns are separated by ||. You can choose any pattern to extract information. 

1. Movie name can be identified as first column in every line formatted as ''[[ <movie name> ]]''  
1. Production/Distribution house is the following column that is formatted as [[< Production house>]]  
1. Director name can be identified with "(director)" tag that follows the name  
1. Genre is present in the next column but make sure to extract only second part that is separated by | operator. For eg. in [Action film|Action] relevant information is Action  
1. Publisher name can be identified in the last column with format "publisher=<publisher name>"  
1. It may help to skip first few lines that contains html code, so that you process actual records.

#### Questions

* Use wrangler to clean the data, then determine which Production/Distribution house produced maximum movies.

#### Notes
* Your wrangler script is not expected to be sophisticated enough to generate the results for the question asked. It just needs to clean/combine the data enough for you to observe the data and answer the question.
* You can export the transformations you carried out in wrangler.
Export the script and paste it in the cell below. Do not bother executing it here in the notebook.
* Stanford also has an online open-source version of Trifacta Wrangler that can be used [here](http://vis.stanford.edu/wrangler/).

In [ ]:
# your wrangler script goes in this cell below
delete row: matches([column1], `{start}!`)
delete row: matches([column1], `\|-`)
delete row: matches([column1], `{start}=`)
delete row: matches([column1], `DO NOT ADD FILMS`)
delete row: matches([column1], `class=|style=|rowspan=`)
delete row: in(sourcerownumber(), [2, 183, 179, 359, 362, 537, 536, 660, 358])
split col: column1 on: '||' limit: 1
replace col: column2 with: '' on: `\| \'\'[[` global: true
replace col: column2 with: '' on: `]]\'\' ` global: true
replace col: column2 with: '' on: `{any}+\|`
replace col: column2 with: '' on: `\|\'\'[[` global: true
rename col: column2 to: 'MovieName'
split col: column3 on: '||' limit: 1
replace col: column1 with: '' on: ` [[` global: true
replace col: column1 with: '' on: `]] ` global: true
replace col: column1 with: '' on: `{start}{delim}`
replace col: column1 with: '' on: `{delim}{end}`
rename col: column1 to: 'ProductionDistributionHouse'
split col: column2 on: `\(director`
replace col: column1 with: '' on: ` [[` global: true
replace col: column1 with: '' on: `]] ` global: true
replace col: column1 with: '' on: `{start}{delim}`
replace col: column1 with: '' on: `{delim}{end}`
rename col: column1 to: 'Director'
split col: column3 on: '||' limit: 1
drop col: column2
split col: column4 on: `\|\|`
replace col: column1 with: '' on: ` [[` global: true
replace col: column1 with: '' on: `]]` global: true
replace col: column1 with: '' on: `{start}{delim}`
replace col: column1 with: '' on: `{delim}{end}`
split col: column1 on: `\|`
replace col: column4 with: '' on: ` film\|{any}+` global: true
set col: column4 value: ifmissing($col, column3)
drop col: column3
rename col: column4 to: 'Genre'
set col: Genre value: ifmissing($col, 'N\/A')
extract col: column2 on: `publisher={any}+ \||publisher={any}+\|`
drop col: column2
replace col: column1 with: '' on: `\|{any}+` global: true
replace col: column1 with: '' on: `{delim}{end}`
replace col: column1 with: '' on: `{delim}{end}`
replace col: column1 with: '' on: `publisher=` global: true
set col: column1 value: ifmissing($col, 'N\/A')
replace col: Director with: '' on: `]]{any}+| \({any}+` global: true
replace col: column1 with: '' on: `[[` global: true
rename col: column1 to: 'Publisher'
replace col: ProductionDistributionHouse with: '' on: `/{any}+|\|{any}+|[[|\({any}+`
replace col: ProductionDistributionHouse with: '' on: `{delim}{end}`

In [ ]:
# wrangler script to get the answer for the question
aggregate value: count() group: ProductionDistributionHouse
sort order: -row_count

### (Question) Which Production/Distribution house produced maximum movies?
#### Warner Bros: 20
#### 20th Century Fox: 14
#### Universal Studios: 14

### worldcup.txt

Use the tool to generate output as follows, i.e., each line in the output contains a country, a year, and the position of the country in that year (if within top 4).

        BRA, 1962, 1
        BRA, 1970, 1
        BRA, 1994, 1
        BRA, 2002, 1
        BRA, 1958, 1
        BRA, 1998, 2
        BRA, 1950, 2
        ...

It may help to 

1. Skip the first 20 or so rows of table headers and other text, so that the data wrangler works with are "record text".  
2. Delete the rows that are clearly HTML formatting content
3. Extract the relevant data from the remaining column into new columns
4. Use the fill operation

#### Questions

* According to the dataset, how often has each country won the world cup?

#### Notes
* Your wrangler script is not expected to be sophisticated enough to generate the results for the question asked. It just needs to clean/combine the data enough for you to observe the data and answer the question.
* You can export the transformations you carried out in wrangler. Export the script and paste it in the cell below. Do not bother executing it here in the notebook.

In [ ]:
# your wrangler script goes in this cell below
delete row: matches([column1], `\|-`)
delete row: in(sourcerownumber(), [1,171,172,173])
replace col: column1 with: '' on: `\|align=center`
replace col: column1 with: '' on: `\|bgcolor=\#FFF68F` global: true
replace col: column1 with: '' on: `[[\#1\|\*]]` global: true
delete row: matches([column1], `{start}\|{digit}+{end}`)
extractlist col: column1 on: `\|{digit}{4}` as: 'Years'
extract col: column1 on: `fb\|{upper}{3}` quote: ''
replace col: column2 with: '' on: `fb\|` global: true
set col: column2 value: fill(column2) order: sourcerownumber()
replace col: Years with: '' on: `\|` global: true
rename col: column2 to: 'Country'
settype col: Years type: 'Array'
derive value: (mod(rownumber(), 5)) - 1 group: Country order: sourcerownumber() as: 'Position'
replace col: Position with: '4' on: `-1` global: true
flatten col: Years
drop col: column1
delete row: ismissing([Years])

In [ ]:
# wrangler script to get the answer for the question
aggregate value: countif(Position == 1) group: Country
sort order: -countif
rename col: countif to: 'Count_of_first_position'

### (Question) how often has each country won the world cup?
#### BRA won the 1st position: 5 times
#### ITA won the 1st position: 4 times
#### GER won the 1st position: 3 times 

***
# 2. Grep, Sed & Awk
***

The set of three UNIX tools, sed, awk, and grep, can be very useful for quickly cleaning up and transforming data for further analysis (and have been around since the inception of UNIX). In conjunction with other unix utilities like sort, uniq, tail, head, etc., you can accomplish many simple data parsing and cleaning tasks with these tools. You are encouraged to play with these tools and familiarize yourselves with the basic usage of these tools.

# grep

```bash
grep 'regexp' filename
```

or equivalently (using UNIX pipelining):

```bash
cat filename | grep 'regexp'
```

The output contains only those lines from the file that match the regular expression. Two options to grep are useful: grep -v will output those lines that do not match the regular expression, and grep -i will ignore case while matching. See the manual (man grep) (or online resources) for more details.

# sed

Sed stands for stream editor. Basic syntax for sed is:

```bash
sed 's/regexp/replacement/g' filename
```

For each line in the intput, the portion of the line that matches regexp (if any) is replaced with replacement. Sed is quite powerful within the limits of operating on single line at a time. You can use \( \) to refer to parts of the pattern match. In the first sed command above, the sub-expression within \( \) extracts the user id, which is available to be used in the replacement as \1.

# awk

Finally, awk is a powerful scripting language (not unlike perl). The basic syntax of awk is:

```bash
awk -F',' 'BEGIN{commands} /regexp1/ {command1} /regexp2/ {command2} END{commands}'
```

For each line, the regular expressions are matched in order, and if there is a match, the corresponding command is executed (multiple commands may be executed for the same line). BEGIN and END are both optional. The -F',' specifies that the lines should be split into fields using the separator ",", and those fields are available to the regular expressions and the commands as $1, $2, etc. See the manual (man awk) or online resources for further details.

# EXAMPLES

#### Note
There's nothing to submit in the examples section. Task to carry out and submit will follow after this section. Just play around and get a hang of the bash commands described here

***

We start off by copying the files from our instabase repository to the VM filesystem our instabse instance is running on.
Remember, you'll have to execute the cell below everytime the VM is restarted (happens when you close and restart the notebook) before you can proceed with the bash examples that follow.

In [ ]:
with ib.open("labor.csv") as f:
    labor=f.read()
with ib.open("crime-clean.txt") as f:
    crime_clean=f.read()
with ib.open("crime-unclean.txt") as f:
    crime_unclean=f.read()

open('/tmp/labor.csv','w').write(labor)
open('/tmp/crime-clean.txt','w').write(crime_clean)
open('/tmp/crime_unclean.txt','w').write(crime_unclean)


A few examples to give you a flavor of the tools and what one can do with them.

* Perform the equivalent of wrap on labor.csv (i.e., merge consecutive groups of lines referring to the same record)

In [ ]:
%%bash 

cat /tmp/labor.csv | awk '/^Series Id:/ {print combined; combined = $0} 
                    !/^Series Id:/ {combined = combined", "$0;}
                    END {print combined}'

*Note that all bash cells begin with **%%bash**. This indicates that, what follows is a bash code/script.*


* On crime-clean.txt, the following command does a fill (first row of output: "Alabama, 2004, 4029.3".

In [ ]:
%%bash 

cat /tmp/crime-clean.txt | grep -v '^,$' | awk '/^[A-Z]/ {state = $4} !/^[A-Z]/ {print state, $0}'
#cat crime-clean.txt | grep -v '^,$' | awk '/^[A-Z]/ {state = $4} !/^[A-Z]/ {print state, $0}'

* On crime-clean.txt, the following script cleans the data. The following works assuming perfectly homogenous data (as the example on the Wrangler website is).

In [ ]:
%%bash

cat /tmp/crime-clean.txt | grep -v '^,$' | sed 's/,$//g; s/Reported crime in //; s/[0-9]*,//' | awk -F',' 'BEGIN {printf "State, 2004, 2005, 2006, 2007, 2008"} /^[A-Z]/ {print c; c=$0} !/^[A-Z]/ {c=c", "$0;} END {print c;}'


* On crime-unclean.txt the follow script perfroms the same cleaning as above, but allows incomplete information (e.g., some years may be missing).

In [ ]:
%%bash

cat /tmp/crime_unclean.txt | grep -v '^,$' | sed 's/Reported crime in //;' | 
        awk -F',' 'BEGIN {printf "State, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008\n"} /^[A-Z]/ || /^$/ { if(state) { printf(state); for(i = 2000; i <= 2008; i++) { if(array[i]) {printf("%s,", array[i])} else {printf("0,")} }; printf("\n");} state=$0; delete array} !/^[A-Z]/ {array[$1] = $2}'

We provided the last example to show how powerful awk can be. However if you need to write a long command like this, you may be better off using a proper scripting language like perl or python!

# Finally, the tasks.

* Use the awk "split" function and "for loop" constructs on World Cup data, to again generate output as follows, i.e., each line in the output contains a country, a year, and the position of the county in that year (if within top 4).

        BRA, 1962, 1
        BRA, 1970, 1
        BRA, 1994, 1
        BRA, 2002, 1
        BRA, 1958, 1
        BRA, 1998, 2
        BRA, 1950, 2
        ...

* Start with the given script that cleans up the data a little bit.
* No need to re-answer the questions in the Wrangler section, but recompute them to ensure your answers are consistent.

In [1]:
with ib.open("worldcup.txt") as f:
    world_cup=f.read()
    
open('/tmp/worldcup.txt','w').write(world_cup)

In [3]:
%%bash 

cat /tmp/worldcup.txt | sed 's/\[\[\([0-9]*\)[^]]*\]\]/\1/g; s/.*fb|\([A-Za-z]*\)}}/\1/g; s/<sup><\/sup>//g; s/|bgcolor[^|]*//g; s/|align=center[^|]*//g; s/:<div[^|]*//g;' |
    grep -v '^|-$' | grep -v '^!' | sed 's/|//g' | 
    awk 'BEGIN {printf "Country\tYear\tPosition\n"} /^[A-Z]/ {country=$0}
        /^[0-9]+ \(.*\)/ || /^ —/ {
            if($0 != " —"){
                split($0,n_year,"\("); 
                split(n_year[2], year_str, "\)"); 
                len = split(year_str[1], year, ", ");
                position = int(NR-1)%6
                for(j=0; j<len; j++){
                    print country"\t"year[j+1]"\t"position
                }   
            }
            #else{ print $0 }
        }' 

Country	Year	Position
BRA	1958	1
BRA	1962	1
BRA	1970	1
BRA	1994	1
BRA	2002	1
BRA	1950	2
BRA	1998	2
BRA	1938	3
BRA	1978	3
BRA	1974	4
ITA	1934	1
ITA	1938	1
ITA	1982	1
ITA	2006	1
ITA	1970	2
ITA	1994	2
ITA	1990	3
ITA	1978	4
GER	1954	1
GER	1974	1
GER	1990	1
GER	1966	2
GER	1982	2
GER	1986	2
GER	2002	2
GER	1934	3
GER	1970	3
GER	2006	3
GER	2010	3
GER	1958	4
ARG	1978	1
ARG	1986	1
ARG	1930	2
ARG	1990	2
URU	1930	1
URU	1950	1
URU	1954	4
URU	1970	4
URU	2010	4
FRA	1998	1
FRA	2006	2
FRA	1958	3
FRA	1986	3
FRA	1982	4
ENG	1966	1
ENG	1990	4
ESP	2010	1
ESP	1950	4
NED	1974	2
NED	1978	2
NED	2010	2
NED	1998	4
TCH	1934	2
TCH	1962	2
HUN	1938	2
HUN	1954	2
SWE	1958	2
SWE	1950	3
SWE	1994	3
SWE	1938	4
POL	1974	3
POL	1982	3
AUT	1954	3
AUT	1934	4
POR	1966	3
POR	2006	4
USA	1930	3
CHI	1962	3
CRO	1998	3
TUR	2002	3
YUG	1930	4
YUG	1962	4
URS	1966	4
BEL	1986	4
BUL	1994	4
KOR	2002	4



# Questions

1. From your experience, briefly discuss the pro and cons between using Data Wrangler as compared to lower levels tools like sed/awk?
2. What additional operations would have made using Data Wrangler "easier"?

#### Note
While responding to markdown cells (as the one below), in case you struggle with formatting, just double click any of the markdown cells in the notebook to see how formatting is done. You may also consult the documentation [here](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html)

***
### Add your response below:

1. Wrangler has a better user interface that is able to interactive with users immediately, but sometimes it works laggily. Moreover, Wrangler use it's own defined regular expressions which should be get used to them, but hopefully, more intuitively. On the other hand, sed/awk use traditional regualr expression that allow us to go into work quickly, but it is somehow quite difficult for person who does not familiar with regular expression. Besdies, sed/awk efficiently deal with all the rows of data simultaneously, but somehow quite hard to understand for a beginner.

2. As mentioned above, Wrangler use it's own defined regular expressions-- such as {any}, {digit}, etc-- that allow users intuitively process the data. Also, Wrangler provides log of commands that allows users to undo or jump back to previous steps quickly.


***
# 3. Tableau
***

Finally, you will perform data exploration with Tableau.

1. Go to the [Tableau Website](https://www.tableau.com/), and download a demo version of Tableau.

    * Tableau gives students a 1 year free license, which can be requested [here](http://www.tableau.com/freeforstudents).

2. Connect Tableau to the the OnTime database, hosted on a postgreSQL server we set up for the course.
To do so, create a new Tableau workbook.
In the workbook, goto data and add a new data source using the credentials below:

    * Type: PostgreSQL (you may have to download an additional driver for postgreSQL, [here](https://www.tableau.com/support/drivers)) (only if you can't connect)
    * Hostname: pg-001.db.gcloud.instabase.com
    * Username: columbia
    * Password: B%38Mt5W@M*QU?Ar
    * Database: db_fea10998_f88d_4b6e_8f90_a6cd73bac65c
    * You should use the table called "Ontime".
    
3. Explore the dataset using Tableau.

The aim of this assignment is to understand (1) which flights are the likeliest to be delayed (2) why they are delayed (3) what we could have missed in the data

**(1) Which flights are delayed? (You're expected to answer any 2 of the 5 questions below)**
- Long flights or short flights?
- Which companies?
- At what time of the day?
- Which state are the most impacted?
- Take California. Which cities are the most concerned? And how about NY state?

**(2) Why are flights delayed? (You're expected to answer any 2  of the 6 questions below)**
- What is the likeliest cause of delays?
- Does that depend on the region?
- Compare California and NY state
- Compare Morning flights and evening flights
- Compare weekends and rest of the week
- Compare first week of dataset and last week

**(3) what we could have missed in the data**
- Find three quirky facts about flight delays. Anything goes, as long it involves at least one aggregate and one filter.

***
### Add you response below:

***
1(a) (Question): Which flights are delayed: Long flights or short flights?

1(a) (Response): According to the definition in https://ipfs.io/ipfs/QmXoypizjW3WknFiJnKLwHCnL72vedxjQkDDP1mXWo6uco/wiki/Flight_length.html, the short flights have length shorter than 3hr (180 mins) and long flights have length greater than 6hr (360 mins). Hence, referring to the size of the bars, we can find that there are more short-hual flights than long-hual flights. However, long-hual flights have higher frequency of delay than other kinds of flights by 6% (Medium-hual) and 10%(Short-hual)

![3.1.png](https://www.dropbox.com/s/vz7gkrwa45up0q6/HW2_3.1.png?raw=1)

***
1(b) (Question): Which companies delayed the most?

1(b) (Response): According to the definition in FFA (https://www.faa.gov/), if flights arrive 15 mins later is delayed; otherwise it's not. From the below graph, we can see that Carrier MQ and F9 have higher frequency of delay. That is up to 30% of their flights arrived later the the plan.

![3.2.png](https://www.dropbox.com/s/q31wcobckcx4mzi/HW2_3.2.png?raw=1)

***
2(a) (Question): Why are flights delayed by comparing California and NY state?

2(a) (Response): According to the size of the circle, we can observe that there are more flights flying to CA than NY. However, the delayed rate of CA is lower than the rate of NY. This refers that different states might affect different delayed rate.

![3.3.png](https://www.dropbox.com/s/k72ac586z3t5dil/HW2_3.3.png?raw=1)

***
2(b) (Question): Why are flights delayed by comparing weekends and rest of the week?

2(b) (Response): According to the 2nd graph below, we can understand that the total number of flights in weekdays is greater than weekends, but through looking at each ratio, we can see that the ratio of weekdays is 26.61% which is about equal to the number of weekend divide to seven days: $\frac{2}{7} = 28.5$%. This means that on average, the total number of flights on each day in the weekends does not have huge different with the one in the weekdays. However, by the 1st graph below, weekends have slightly higher rate of delay than weekdays though there is no huge difference on the number of flights each day. Hence, the day of the flights might affect the rate of delay.

![3.3.png](https://www.dropbox.com/s/uyn9xfpyvfz7iky/HW2_3.4.png?raw=1)

***
3 (Response): There are many factors can be considered while discussing flight delays. As discussed above, locations, days (time), type of flights, and carriers. By aggregating and filtering the delay, we might lose some information during the process. 

 1) When we group by the day of the week and filter by `Arrdelaytime`, we will easily miss the data of different carriers. Moreover, the Deptime and Arrtime seems to be incomprehensible and there are some missing data, either.
    
 2) While we consider the location (`Desstate`) and filter by `isDelay` (`Arrdel15`), we will miss the details about time of the flights; also, it is difficult for us to determine what the relationship is between origin and destination causing delay.
    
 3) As our discussion in Q1.(a), we understand that long-hual flights have higher probability to delay than short or medium-hual flights. However, there are only some specific carriers that offer long-hual flights. This refers that the carriers providing long-hual flights might have higher delayed rate than other carriers that only provide short or medium-hual flights. Thus, when we group by different carriers, we might mis-summarize that some companies delay more than the others. 



# Submission

* Got to 'File' and download this notebook as .ipynb
* Rename it as **data\_processing\_[your uni].ipynb**
* Then submit it on courseworks
